<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/notebooks/main_tests_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Handle importing/installing, both local and on Colab**

In [1]:
import sys
ON_COLAB = 'google.colab' in sys.modules
if ON_COLAB:
    # Do stuff that only needs to happen on colab
    !pip install pyspark  # noqa
    !pip install ijson
    !pip install ipython-autotime
    %load_ext autotime
    pass
else:
    # Do stuff that only needs to happen on local computer
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=151647b14417b4b31d2d55a434079d267d0591afe9197fd67cc8cc049c36a5f8
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 10.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.9 MB/s eta 0:00:00
time: 330 µs (started: 2023-06-17 11:46:47 +00:00)


Rerun the code cell below to use the latest version of the python files!

Test all functions you want inside the run_all_tests() during development, for small sample sizes.


Code below is for actual result generation later, so that we can easily reuse intermediate values.

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/MyDrive/colab_notebooks/data_intensive_systems/src')

from pyspark.sql import SparkSession

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.appName("Clustering").getOrCreate()

In [8]:
#DEFINING THE PARSE FUNCTIONS

import json
import ijson
import pandas as pd
from pyspark.sql.functions import collect_list, udf
from pyspark.sql import SparkSession
from pyspark.sql.types import MapType, IntegerType, StringType, FloatType
import warnings
import math
from pyspark.sql.types import StructType, StructField, MapType, StringType, IntegerType, ArrayType
import pyspark.sql.functions as F
from pyspark.ml.linalg import SparseVector, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StructType, StructField, StringType, FloatType

def parse_nested_data(json_path, debug_flag=False):
    """
    Parse the data from the json file to a pandas df.
    """
    warnings.filterwarnings("ignore", category=FutureWarning)
    num_routes = 0
    from_tos = set()
    products = set()
    with open(json_path, 'rb') as f:
        for row in ijson.items(f, "item"):
            num_routes += 1
            for trip in row['route']:
              from_to = trip['from']+"-"+trip['to']
              from_tos.add(from_to) #does not add duplicate from_to's
              for prod in trip["merchandise"]:
                products.add(prod)
    from_tos = list(from_tos)
    products = list(products)
    #Integers relate to specific products.
    #For bookkeeping we store a dictionary which store the index of a specific product.
    product_mapping = {}
    for i,v in enumerate(products):
      product_mapping[v] = i

    df_rows = []
    with open(json_path, 'rb') as f:
      for row in ijson.items(f, "item"):
        new_row = {"id-sr": str(row["id"])+" "+ str(row["sr"])}
        for trip in row['route']:
            from_to = trip['from']+"-"+trip['to']
            merch_dict = dict(map(lambda x: (product_mapping[x[0]], x[1]), trip["merchandise"].items()))
            trip_merch = {from_to: merch_dict}
            new_row.update(trip_merch)
        df_rows.append(new_row)
    df = pd.DataFrame(df_rows, columns=["id-sr"] +from_tos)
    df = df.applymap(lambda x: {} if pd.isna(x) else x)
    return df, from_tos, products, num_routes

def get_nested_data(spark, path, clustering_settings):
  """
  Reads in the data in a more human readable format. Every route has multiple columns, each representing a trip. The value for every trip is a (sparse) dictionary containing the products for that trip.
  These routes fit into the custom created distance function which might be a good way to assign costs to routes depending on how much they differ from their cluster centres.
  """
  df, from_tos, products, num_routes = parse_nested_data(path, clustering_settings["debug_flag"])
  schema = StructType([StructField("id-sr",StringType())] + [StructField(from_to, MapType(IntegerType(), IntegerType())) for from_to in from_tos]  )
  spark_df = spark.createDataFrame(df, schema = schema)
  if clustering_settings["debug_flag"]:
    spark_df.show()
  clustering_settings["num_routes"] = num_routes
  return spark_df.rdd


def get_vector_dataframe(spark, json_path, debug_flag=False):
  """
  Functions that reads in the row's of the data and stores all the features in a sparse vector (similar to a dict). A feature of a route is defined by (from city, to city, product).
  Every feature is represented by an integer key and the value is the amount of product.
  A row in the resulting dataframe has the following structure:
  Row(id-sr='0 0', features=SparseVector(38466, {237: 10.0, 4952: 14.0, 6185: 6.0, 9978: 15.0, 10914: 19.0, 11111: 9.0, 11429: 6.0, 12992: 10.0, 13258: 7.0, 15131: 8.0, 15287: 9.0, 17500: 7.0, 18674: 6.0, 18898: 19.0, 19172: 17.0, 20302: 13.0, 20366: 11.0, 20738: 6.0, 21733: 7.0, 23069: 11.0, 23793: 7.0, 26398: 6.0, 26739: 12.0, 27197: 13.0, 29726: 7.0, 30636: 16.0, 32340: 17.0, 33394: 7.0, 33651: 19.0, 33868: 7.0, 33972: 12.0}))
  The function also returns a dict that translates the sparse vector keys back as follows: 237: 'city_234-city_130-product_136'
  """
  num_routes = 0
  from_to_prods = set()
  with open(json_path, 'rb') as f:
      for row in ijson.items(f, "item"):
          num_routes += 1
          for trip in row['route']:
            for prod in trip["merchandise"]:
              from_to_prods.add(trip['from']+"-"+trip['to']+"-" + prod)
  from_to_prods = list(from_to_prods)
  indices2from_to_prods = {idx: ft_prod for idx,
                           ft_prod in enumerate(from_to_prods)}
  from_to_prods2indices = {ft_prod: idx for idx,
                           ft_prod in enumerate(from_to_prods)}

  sparse_vectors = []
  with open(json_path, 'rb') as f:
      for row in ijson.items(f, "item"):
          route_sparse_dict = {}
          for trip in row['route']:
              for prod, val in trip['merchandise'].items():
                  from_to_prod = from_to_prods2indices[trip['from'] +
                                                       "-" + trip['to'] + "-" + prod]
                  if from_to_prod in route_sparse_dict:
                      route_sparse_dict[from_to_prod] += val
                  else:
                      route_sparse_dict[from_to_prod] = val
          sparse_vectors.append(
              (str(row['id']) + " " + str(row['sr']), dict(sorted(route_sparse_dict.items()))))

  sparse_vectors_with_indices = [
      (id_sr, SparseVector(len(from_to_prods), list(
          route_dict.keys()), list(route_dict.values())))
      for id_sr, route_dict in sparse_vectors
  ]

  schema = StructType([StructField("id-sr", StringType()),
                       StructField("features", VectorUDT())])
  spark_df = spark.createDataFrame(sparse_vectors_with_indices, schema=schema)

  if debug_flag:
      print("Dataset structure")
      spark_df.show(5)
      print("What a single element looks like:")
      print(spark_df.head())
      print("indices 2 from_to_prod:", indices2from_to_prods)
  return spark_df, indices2from_to_prods



In [5]:
clustering_settings = {
    'clustering_algorithm': 'kmodes',
    'k_values': [10],
    'max_iterations': 20,
    'debug_flag': True
}

In [9]:
spark_df, indices2from_to_prods = get_vector_dataframe(spark, "/content/drive/MyDrive/colab_notebooks/data_intensive_systems/data/100000_0.500_actual_routes.json", clustering_settings)

Dataset structure
+-----+--------------------+
|id-sr|            features|
+-----+--------------------+
|  0 0|(38466,[1510,2437...|
|  1 0|(38466,[1510,2437...|
|  2 0|(38466,[1510,3712...|
|  3 0|(38466,[1510,2437...|
|  4 0|(38466,[2437,4222...|
+-----+--------------------+
only showing top 5 rows

What a single element looks like:
Row(id-sr='0 0', features=SparseVector(38466, {1510: 8.0, 2437: 15.0, 3712: 7.0, 4222: 11.0, 5944: 7.0, 5992: 7.0, 6168: 6.0, 6345: 6.0, 9579: 10.0, 10273: 17.0, 10896: 16.0, 11972: 11.0, 12540: 6.0, 12707: 9.0, 12958: 7.0, 15238: 6.0, 15731: 12.0, 16284: 7.0, 20261: 12.0, 20644: 6.0, 20888: 7.0, 23047: 19.0, 23354: 10.0, 23565: 13.0, 27073: 13.0, 29754: 19.0, 35701: 19.0, 36109: 17.0, 36189: 7.0, 36893: 9.0, 37587: 14.0}))
indices 2 from_to_prod: {0: 'city_68-city_234-product_367', 1: 'city_159-city_68-product_371', 2: 'city_168-city_16-product_840', 3: 'city_178-city_13-product_25', 4: 'city_190-city_9-product_877', 5: 'city_0-city_107-product_58', 6: 

In [21]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, substring,  countDistinct, count

# Extracting the feature vectors without the ID column
feature_vectors = spark_df.select('features')

# Creating the K-means instance
kmeans = KMeans(k=10)

# Training the K-means model
model = kmeans.fit(feature_vectors)

# Getting the cluster predictions
clustered_df = model.transform(spark_df)

if clustering_settings["debug_flag"]:
  print("clustered_df structure:")
  clustered_df.show(5)

clustered_df structure:
+-----+--------------------+----------+
|id-sr|            features|prediction|
+-----+--------------------+----------+
|  0 0|(38466,[1510,2437...|         7|
|  1 0|(38466,[1510,2437...|         7|
|  2 0|(38466,[1510,3712...|         7|
|  3 0|(38466,[1510,2437...|         7|
|  4 0|(38466,[2437,4222...|         7|
+-----+--------------------+----------+
only showing top 5 rows



In [22]:
#Generate a table which has the standard routes as rows and the cluster centres as columns showing which routes (grouped by the sr they belong to) are assigned to which cluster centre.
grouped_df = clustered_df.withColumn("sr\predicted_clusters", col("id-sr").substr(-1, 1)) \
    .groupBy("sr\predicted_clusters") \
    .pivot("prediction") \
    .agg(count("*").alias("count"))

grouped_df.show()

+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|sr\predicted_clusters|    0|    1|    2|    3|    4|    5|    6|    7|    8|    9|
+---------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|                    7| null| null| null| null| null| null|10000| null| null| null|
|                    3| null|10000| null| null| null| null| null| null| null| null|
|                    8| null| null| null| null| null| null| null| null| null|10000|
|                    0| null| null| null| null| null| null| null|10000| null| null|
|                    5| null| null|10000| null| null| null| null| null| null| null|
|                    6|10000| null| null| null| null| null| null| null| null| null|
|                    9| null| null| null| null|10000| null| null| null| null| null|
|                    1| null| null| null|10000| null| null| null| null| null| null|
|                    4| null| null| null| null| null|10000| null| null| null

In [23]:
#EVELUATE THE CLUSTERING FOR DIFFERENT K VALUES AND DETERMINE THE BEST K.
#TODO: Automate this.


evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(clustered_df)
print("Silhouette with squared euclidean distance = " + str(silhouette))

#k=11 gives 0.8593
#k=10 gives sillhouete 0.8917
#k= 9 gives 0.805
#So this is probably a good way to determine the best k value


Silhouette with squared euclidean distance = 0.8917156406590634


In [38]:
#COMPARE THE FOUND CLUSTER CENTERS TO THE STANDARD ROUTES
#TODO: Make the comparison more concrete. Also use distances.

def parse_center(cluster_center):
  rep_cluster= {}
  cluster_center = {indices2from_to_prods[i]:v for i,v in enumerate(cluster_center) if v >0.5}
  for key,value in cluster_center.items():
    split = key.split("-")
    from_to = split[0] + "-" + split[1]
    prod = split[2]
    if from_to in rep_cluster:
        rep_cluster[from_to].update({prod: value})
    else:
        rep_cluster[from_to] = {prod: value}
  return rep_cluster

cluster_centers = model.clusterCenters()
parsed_cluster_centers = [parse_center(center) for center in cluster_centers]

# standard route 6 is given by: {"id":6, "route":[{"from":"city_242","to":"city_31","merchandise":{"product_138":17,"product_406":10,"product_367":6,"product_427":14,"product_237":11}},{"from":"city_31","to":"city_53","merchandise":{"product_74":15,"product_603":8,"product_788":14}},{"from":"city_53","to":"city_99","merchandise":{"product_150":10,"product_977":16,"product_933":18,"product_158":5,"product_529":13}},{"from":"city_99","to":"city_111","merchandise":{"product_262":7,"product_548":12,"product_695":19}},{"from":"city_111","to":"city_32","merchandise":{"product_611":7,"product_165":5,"product_341":15}},{"from":"city_32","to":"city_126","merchandise":{"product_107":14,"product_194":18,"product_248":8,"product_437":12}},{"from":"city_126","to":"city_157","merchandise":{"product_814":16,"product_532":14,"product_314":14,"product_414":9}},{"from":"city_157","to":"city_53","merchandise":{"product_263":8,"product_703":7,"product_925":17,"product_457":9,"product_226":10}}]},
# The first center is given by: {'city_126-city_157': {'product_414': 8.6262, 'product_314': 13.4242, 'product_532': 13.4267, 'product_814': 15.3308}, 'city_157-city_53': {'product_457': 8.592600000000001, 'product_226': 9.538, 'product_263': 7.6362000000000005, 'product_925': 16.2176, 'product_703': 6.674300000000001}, 'city_32-city_126': {'product_248': 7.523000000000001, 'product_437': 11.264000000000001, 'product_107': 13.1304, 'product_194': 16.8872}, 'city_53-city_99': {'product_977': 14.965100000000001, 'product_933': 16.8401, 'product_150': 9.3587, 'product_158': 4.6893, 'product_529': 12.1645}, 'city_242-city_31': {'product_406': 9.595600000000001, 'product_138': 16.297900000000002, 'product_237': 10.5474, 'product_427': 13.424100000000001, 'product_367': 5.766100000000001}, 'city_99-city_111': {'product_262': 6.5544, 'product_695': 17.7737, 'product_548': 11.2245}, 'city_31-city_53': {'product_74': 14.3298, 'product_603': 7.6515, 'product_788': 13.3873}, 'city_111-city_32': {'product_611': 6.5434, 'product_165': 4.6871, 'product_341': 14.0013}}

{'city_126-city_157': {'product_414': 8.6262, 'product_314': 13.4242, 'product_532': 13.4267, 'product_814': 15.3308}, 'city_157-city_53': {'product_457': 8.592600000000001, 'product_226': 9.538, 'product_263': 7.6362000000000005, 'product_925': 16.2176, 'product_703': 6.674300000000001}, 'city_32-city_126': {'product_248': 7.523000000000001, 'product_437': 11.264000000000001, 'product_107': 13.1304, 'product_194': 16.8872}, 'city_53-city_99': {'product_977': 14.965100000000001, 'product_933': 16.8401, 'product_150': 9.3587, 'product_158': 4.6893, 'product_529': 12.1645}, 'city_242-city_31': {'product_406': 9.595600000000001, 'product_138': 16.297900000000002, 'product_237': 10.5474, 'product_427': 13.424100000000001, 'product_367': 5.766100000000001}, 'city_99-city_111': {'product_262': 6.5544, 'product_695': 17.7737, 'product_548': 11.2245}, 'city_31-city_53': {'product_74': 14.3298, 'product_603': 7.6515, 'product_788': 13.3873}, 'city_111-city_32': {'product_611': 6.5434, 'product_

In [ ]:


print(parsed_cluster_centers[0])

In [17]:
#THIS IS NO LONGER USED IN THE CURRENT WORKFLOW BUT MIGHT CONTAIN SOME USEFUL FUNCTION FOR EVALUATION ETC

from copy import copy
from statistics import mode
from pyspark import RDD
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from functools import reduce

import numpy as np
import math
from collections import Counter
import random

def run_clustering(clustering_settings: dict, data: RDD) -> list[tuple]:
    '''Define variables to store results.'''
    # E.g. for kmodes: [(predicted_centroids, (k, init_mode)), ...]
    results = []

    # Check which clustering algortihm to run
    if clustering_settings['clustering_algorithm'] == 'kmodes':
        for current_k in clustering_settings['k_values']:
            # TODO in the future add other parameters here.
            # Run clustering with current parameters
            print("Performing clustering with k= ", current_k)
            predicted_centroids = kModes(
                data=data,
                k=current_k,
                clustering_settings=clustering_settings
            )

            # Store the settings, model, and metrics
            results.append((predicted_centroids, {'k': current_k}))
            if clustering_settings["debug_flag"]:
                print("The centroids for  k = ", current_k, " are given by: ", [c[0] for c in predicted_centroids] )
    else:
        print("Clustering algorithm setting not recognized in run_and_tune().")
    if clustering_settings["debug_flag"]:
        print("The output results for multiple k is given by:", results)
    return results


def kModes(data: RDD, k: int, clustering_settings):
    # Painfull code duplication which is the only way I managed to make all the spark dependencies work
    def dictionary_distance(dict1, dict2):
        # This function computes the normalized euclidean distance (in 0-1) for dict representations of (sparse) vectors.
        norm_dict1 = math.sqrt(np.sum(
            [int(float(v)) ** 2 for k, v in dict1.items()]))
        norm_dict2 = math.sqrt(np.sum(
            [int(float(v)) ** 2 for k, v in dict2.items()]))
        return math.sqrt(np.sum(
            [(int(float(dict1.get(product, 0))) - int(float(dict2.get(product, 0)))) ** 2 for product in
             set(dict1) | set(dict2)])) / (norm_dict1 + norm_dict2)

    def route_distance(route1, route2):
      columns = route1.__fields__[1:]
      intersection = 0
      union = 0
      intersecting_dist = 0
      # Preferably vectorize this, something with zip?
      for column in columns:
          trip1 = route1[column]
          trip2 = route2[column]
          if trip1 or trip2:
              union += 1
              if trip1 and trip2:
                  intersection += (1 - dictionary_distance(trip1, trip2))
      if union != 0:
          dist = 1 - intersection / union
      else:
          dist = 1
      return dist


    def assign_row_to_centroid_key(row, centroids):
      random_centroid = random.choice(centroids)
      min_centroid = min(centroids, key=lambda centroid: route_distance(row, centroid))
      if route_distance(row, random_centroid) == route_distance(row, min_centroid):
          return (random_centroid["id-sr"], row)
      else:
          return (min_centroid["id-sr"], row)

    def create_centroid(set_of_rows):
        size_of_set = len(set_of_rows)
        trips_to_keep = []
        first_row = True
        for row in set_of_rows:
            if first_row:
                trips_to_keep = np.zeros(len(row))
                first_row = False
            for it, trip in enumerate(row):
                if trip:
                    trips_to_keep[it] += 1
        trips_to_keep = trips_to_keep >= size_of_set // 2
        row_scores = []
        for row in set_of_rows:
            row_score = 0
            for it, trip in enumerate(row):
                if it != 0 and trip and trips_to_keep[it]:
                    row_score += 1
            row_scores.append(row_score)
        max_score = -1
        for it, row in enumerate(set_of_rows):
            if row_scores[it] > max_score:
                best_row = row
                max_score = row_scores[it]
            if row_scores[it] == max_score:
              if random.random() <= 0.2:
                best_row = row

        # if clustering_settings["debug_flag"]:
        #   print("trips_to_keep", [int(bl) for bl in trips_to_keep])
        #   print("row_scores", row_scores)

        return best_row

    centroids = data.takeSample(withReplacement=False, num=k)
    if clustering_settings["debug_flag"]:
        print("centroids = ",  [c[0] for c in centroids])

    # Iterate until convergence or until the maximum number of iterations is reached
    for i in range(clustering_settings["max_iterations"]):
        if clustering_settings["debug_flag"]:
          print("iteration ", i, ": ")
        # Assign each point to the closest centroid
        clusters = data.map(lambda row: assign_row_to_centroid_key(row, centroids)).groupByKey()

        if clustering_settings["debug_flag"]:
          print("Mapped rows to existing centroids")
          ls_set_of_rows = list(clusters.take(k))
          for i in range(len(ls_set_of_rows)):
            print("number of rows in the", i, "-th cluster per st route:" , Counter([row_[0].split()[1] for row_ in ls_set_of_rows[i][1]]) )
          print("Computing the new centroid for the first cluster:")
          print("new_c= ", create_centroid(ls_set_of_rows[0][1]))

        centroids = clusters.map(lambda key_rows: create_centroid(key_rows[1])).collect()

        if clustering_settings["debug_flag"]:
            print("centroids = ",  [c[0] for c in centroids])

    return [list(x) for x in centroids]